<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Vector Representations
## *Data Science Unit 4 Sprint 2 Assignment 2*

In [1]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy

from bs4 import BeautifulSoup

## 1) *Clean:* Job Listings from indeed.com that contain the title "Data Scientist" 

You have `job_listings.csv` in the data folder for this module. The text data in the description column is still messy - full of html tags. Use the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library to clean up this column. You will need to read through the documentation to accomplish this task. 

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

import spacy
nlp = spacy.load("en_core_web_lg")



df= pd.read_csv('data/job_listings.csv')  
df.head()

         



,Unnamed: 0,description,title
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist


In [3]:
df['description'].head(23)

0     b"<div><div>Job Requirements:</div><ul><li><p>...
1     b'<div>Job Description<br/>\n<br/>\n<p>As a Da...
2     b'<div><p>As a Data Scientist you will be work...
3     b'<div class="jobsearch-JobMetadataHeader icl-...
4     b'<ul><li>Location: USA \xe2\x80\x93 multiple ...
5     b'<div>Create various Business Intelligence An...
6     b'<div><p>As Spotify Premium swells to over 96...
7     b"Everytown for Gun Safety, the nation's large...
8     b"<ul><li>MS in a quantitative discipline such...
9     b'<div><p>Slack is hiring experienced data sci...
10    b'<div><p><b>Who We Are</b></p><p>\nBlackThorn...
11    b'<div class="jobsearch-JobMetadataHeader icl-...
12    b"nfosys\xe2\x80\x93 Data &amp; Analytics \xe2...
13    b'<div><p>As Spotify Premium swells to over 96...
14    b'<div><ul><li>Experience with guiding R&amp;D...
15    b'<div><p>The Atlantic is seeking a Data Scien...
16    b'<div><p><b>THE CHALLENGE</b></p><p>\nEventbr...
17    b'<div class="jobsearch-JobMetadataHeader 

## 2) Use Spacy to tokenize the listings 

In [19]:
from ftfy import fix_encoding, fix_text
def get_text(desc):

    soup = BeautifulSoup(desc)

    return fix_text(fix_encoding(soup.text))

df['description'] = df['description'].apply(get_text)
df.head(25)

,Unnamed: 0,description,title
0,0,"b""Job Requirements:\nConceptual understanding ...",Data scientist
1,1,"b'Job Description\n\nAs a Data Scientist 1, yo...",Data Scientist I
2,2,b'As a Data Scientist you will be working on c...,Data Scientist - Entry Level
3,3,"b'$4,969 - $6,756 a monthContractUnder the gen...",Data Scientist
4,4,b'Location: USA \xe2\x80\x93 multiple location...,Data Scientist
5,5,b'Create various Business Intelligence Analyti...,Data Scientist
6,6,b'As Spotify Premium swells to over 96M subscr...,Associate Data Scientist – Premium Analytics
7,7,"b""Everytown for Gun Safety, the nation's large...",Data Scientist
8,8,"b""MS in a quantitative discipline such as Stat...",Sr. Data Scientist
9,9,b'Slack is hiring experienced data scientists ...,"Data Scientist, Lifecyle"


In [16]:


def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [23]:
df['tokenize_description'] = df['description'].apply(tokenize)
df.head()


,Unnamed: 0,description,title,tokenize_description
0,0,"b""Job Requirements:\nConceptual understanding ...",Data scientist,"[b""Job, requirements:\nconceptual, understandi..."
1,1,"b'Job Description\n\nAs a Data Scientist 1, yo...",Data Scientist I,"[b'Job, description\n\na, Data, scientist, 1, ..."
2,2,b'As a Data Scientist you will be working on c...,Data Scientist - Entry Level,"[b'As, Data, scientist, work, consult, busines..."
3,3,"b'$4,969 - $6,756 a monthContractUnder the gen...",Data Scientist,"[b'$4,969, $, 6,756, monthcontractunder, gener..."
4,4,b'Location: USA \xe2\x80\x93 multiple location...,Data Scientist,"[b'Location, USA, \xe2\x80\x93, multiple, loca..."


## 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

# list of text documents
target = df['description']

# create the transformer
vect = CountVectorizer()

# build vocab
vect.fit(target)

# transform text
dtm = vect.transform(target)


In [46]:
#print(vect.get_feature_names())

In [51]:
target[:426]

0      b"Job Requirements:\nConceptual understanding ...
1      b'Job Description\n\nAs a Data Scientist 1, yo...
2      b'As a Data Scientist you will be working on c...
3      b'$4,969 - $6,756 a monthContractUnder the gen...
4      b'Location: USA \xe2\x80\x93 multiple location...
                             ...                        
421    b"About Us:\nWant to be part of a fantastic an...
422    b'InternshipAt Uber, we ignite opportunity by ...
423    b'$200,000 - $350,000 a yearA million people a...
424    b"SENIOR DATA SCIENTIST\nJOB DESCRIPTION\n\nAB...
425    b'Cerner Intelligence is a new, innovative org...
Name: description, Length: 426, dtype: object

In [47]:
#print(dtm)

In [45]:
dtm.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 2, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [48]:
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())
dtm

,00,000,02115,03,0356,04,062,06366,08,10,...,zenreach,zero,zeus,zf,zheng,zillow,zones,zoom,zuckerberg,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
423,0,2,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
424,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
len(dtm)

426

## 4) Visualize the most common word counts

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(stop_words='english', min_df=2, max_df=.95, ngram_range=(1,2))

#Learn our Vocab
vect.fit(target)

# Get sparse dtm
dtm = vect.transform(target)

dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())


In [58]:
print(f"number of features (ngrams): {dtm.shape[1]}")

number of features (ngrams): 29298


In [59]:
dtm.head()

,000,000 125,000 350,000 associates,000 cities,000 client,000 employees,000 nthe,000 restaurants,000 subcontractor,...,zero,zero invite,zf,zf digital,zf divisions,zf organization,zf platforms,zf portfolio,zf positioning,zf xe2
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
row = dtm.mean(axis=1)
row.head(20)

0     0.003959
1     0.017476
2     0.003993
3     0.006246
4     0.001263
5     0.009967
6     0.013721
7     0.024985
8     0.013960
9     0.019250
10    0.013346
11    0.025087
12    0.032937
13    0.013721
14    0.018841
15    0.019967
16    0.016895
17    0.003140
18    0.024985
19    0.002833
dtype: float64

## 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Create a vocabulary and get word counts per document
# Similiar to fit_predict
dtm = tfidf.fit_transform(target)

# Print word counts

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,000,04,10,100,1079302,11,12,125,14,15,...,years,yearthe,yes,yeti,york,young,yrs,zeus,zf,zillow
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.093431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
# Tunning Parameters

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', 
                        ngram_range=(1,2),
                        max_df=.97,
                        min_df=3,
                        tokenizer=tokenize,
                        max_features=5000)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(target) # Similiar to fit_predict

# Print word counts

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,$,+,+ year,+ years\xe2\x80\x99,0,0 2,1,1 year,10,10 year,...,york city,you\'ll,you\xe2\x80\x99ll,you\xe2\x80\x99ll work,you\xe2\x80\x99re,you\xe2\x80\x99re look,you\xe2\x80\x99ve,yrs,|,||
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.030238,0.033017,0.0,0.0,0.0,0.114019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.194643,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.135861,0.148350,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [63]:
from sklearn.neighbors import NearestNeighbors

# Fit on DTM
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [65]:
nn.kneighbors([dtm.iloc[23].values])

(array([[0.        , 0.        , 1.25036249, 1.26606172, 1.26622376]]),
 array([[ 23, 123, 256, 371, 337]], dtype=int64))

In [66]:
target[256][:200]

"b'Role Summary:\\nThe CCS Data Scientist is responsible for supporting the CCS digital service operations teams with data analytics that drive improved remote service efficiency and improved customer e"

In [67]:
ideal =  [ """ Machine learning position in Silicon Valley, preferably paying a lot of money and in a very nice warm climate """]

In [68]:
new = tfidf.transform(ideal)

In [69]:
nn.kneighbors(new.todense())

(array([[1.32987623, 1.33944941, 1.35263427, 1.35801671, 1.36013465]]),
 array([[364,  84, 287, 151, 315]], dtype=int64))

In [70]:
target[364]

'b\'CircleUp harnesses the power of machine learning and predictive analytics to discover some of the fastest-growing companies in the consumer & retail sector. Our mission is to help entrepreneurs thrive by giving them the resources and capital they need. We are building a predictive data system called "Helio" to bring the data-driven revolution that has occurred in the public markets to the private markets, starting with consumer & retail.\\nWe are working on challenging problems in information retrieval, entity resolution, and machine learning. We are developing an in-depth knowledge graph of all private companies by mining vast amounts of data to successfully rewrite the rules on how private companies are evaluated.\\nWe are seeking a data scientist who will work as a member of our Insights team delivering insights to entrepreneurs. Partnering closely with our business team members, you will work directly with Helio data and artifacts, developing new insights and visualizations to 

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.
 - Create a labeled dataset - which jobs will you apply for? Train a model to select the jobs you are most likely to apply for. :) 